In [291]:
%load_ext autoreload
%autoreload 2
import numpy as np
import pandas as pd
from src.features.conferences import ConferenceFeatures
from src.utils import load_data_template


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [292]:
data = load_data_template(season=False)
data.dropna(subset=['a_win'], inplace=True)
conf_feat = ConferenceFeatures(default_lags=0)
data = conf_feat.per_team_wrapper(data, conf_feat.conference_games, per_game=False, per_day=False)
data.dropna(inplace=True)
print(data.shape)
data.sample()

(2117, 9)


,Season,team_a,team_b,in_target,game_set,a_win,DayNum,conference_team_a,conference_team_b
9814,1996,1280,1433,False,0,True,136,42,14


In [293]:
data.columns

Index(['Season', 'team_a', 'team_b', 'in_target', 'game_set', 'a_win',
       'DayNum', 'conference_team_a', 'conference_team_b'],
      dtype='object')

In [294]:
data.conference_team_b

15       42
31       36
130      11
148      50
176      34
192       9
195      11
247      12
268       7
293      11
322      11
362      39
435      47
444      31
459      25
559       3
601       3
630      42
635      12
638      42
645      11
662       1
673       7
727      46
741      42
765      11
775      39
776       3
810      11
861       0
         ..
10931     7
10932     7
10933     1
10934    11
10935     1
10936     7
10937    31
10938     3
10939    34
10940    11
10941     1
10942     3
10943    42
10944     3
10945    11
10946     3
10947    11
10948     7
10949    31
10950    11
10951    42
10952    39
10953    11
10954     7
10955    31
10956     7
10957    11
10958    31
10959     7
10960    11
Name: conference_team_b, Length: 2117, dtype: int64

In [296]:
from xgboost import XGBClassifier
from sklearn.model_selection import KFold
from sklearn.metrics import log_loss


X = data.drop(['Season', 'team_a', 'team_b', 'a_win', 'in_target', 'DayNum'], axis=1)
y = data[['a_win']].astype(int)

kf = KFold(n_splits=5, shuffle=True)
metrics = []
for tr_i, t_i in kf.split(X):
    X_tr, y_tr = X.iloc[tr_i], y.iloc[tr_i]
    X_t, y_t = X.iloc[t_i], y.iloc[t_i]
    xgb = XGBClassifier()
    xgb.fit(X_tr.values, y_tr.values.reshape(-1))
    preds = xgb.predict_proba(X_t.values)
    metric = log_loss(y_t.values, preds)
    print(metric)
    metrics.append(metric)
    
print('Metric Mean: {:.2f} ({:.2f})'.format(np.mean(metrics), np.std(metrics)))

0.641950120148
0.628830014921
0.61431774136
0.638484151492
0.625241513476
Metric Mean: 0.63 (0.01)
